In [1]:
using PCT
using Revise

Precompiling PCT
  ✓ PCT
  1 dependency successfully precompiled in 2 seconds. 7 already precompiled.


In [5]:
f, _ = @pct (A::Her) -> pullback((x::CV) -> sum((i, j), x(i)' * A(i, j) * x(j))); f

(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(i)'⋅x(j)⋅A(i, j)))

In [6]:
res = blaserize(f)

1 conjugate call
(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(i)'⋅x(j)⋅A(i, j)))
Map
(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(j)⋅A(i, j)⋅x⁺(i)))
-->

2 blas mul
(A) -> 
    𝒫((x) -> 
        ∑((i, j), x(j)⋅A(i, j)⋅x⁺(i)))
Map
(A) -> 
    𝒫((x) -> 
        ∑((j), (x⁺⋅A)(j)⋅x(j)))
-->

3 blas mul
(A) -> 
    𝒫((x) -> 
        ∑((j), (x⁺⋅A)(j)⋅x(j)))
Map
(A) -> 
    𝒫((x) -> 
        x⁺⋅A⋅x)
-->



(A) -> 
    𝒫((x) -> 
        x⁺⋅A⋅x)

In [7]:
df = redux(vdiff(f), settings=symmetry_settings) |> blaserize # |> latex |> println

1 sum_dist
∑((i, j), ((_d) -> 
    δ(i, _d, x(j)⋅A(i, j))+(_d) -> 
    ∑((_z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)'))))
Add
((_d) -> 
    ∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+(_d) -> 
    ∑((i, j, _z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)')))
-->

2 combine map
((_d) -> 
    ∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+(_d) -> 
    ∑((i, j, _z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)')))
Map
(_d) -> 
    (∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+∑((i, j, _z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)')))
-->

3 contract_delta
(_d) -> 
    (∑((i, j), δ(i, _d, x(j)⋅A(i, j)))+∑((i, j, _z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)')))
Map
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑((i, j, _z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)')))
-->

4 contract_delta
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑((i, j, _z_1), δ(j, _d, δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, j)')))
Map
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑((_z_1, i), δ(x(i)', _z_1, 1)⋅_z_1'⋅A(i, _d)'))
-->

5 swallow
(_d) -> 
    (∑((j), x(j)⋅A(_d, j))+∑(

(A) -> 
    (x) -> 
        2.0⋅A⋅x

In [11]:
f, _ = @pct (A::Sym, r::RV, p::RV, b::RV, x::RV) -> begin
    E = (x::RV) -> sum((i, j), 0.5 * x(i) * A(i, j) * x(j)) - sum(i, x(i) * b(i))
    pullback((a::R, b::R) -> E((i::N) -> x(i) + a * (r(i) + b * p(i))))
end; blaserize(f)

1 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i, j), 0.5⋅x(i)⋅x(j)⋅A(i, j))-1.0⋅∑((i), x(i)⋅b(i)))
        𝒫((a, b) -> 
            E((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i))))
    end
Map
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((j), 0.5⋅(xᵀ⋅A)(j)⋅x(j))-1.0⋅∑((i), x(i)⋅b(i)))
        𝒫((a, b) -> 
            E((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i))))
    end
-->

2 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((j), 0.5⋅(xᵀ⋅A)(j)⋅x(j))-1.0⋅∑((i), x(i)⋅b(i)))
        𝒫((a, b) -> 
            E((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i))))
    end
Map
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅∑((i), x(i)⋅b(i))+0.5⋅xᵀ⋅A⋅x)
        𝒫((a, b) -> 
            E((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i))))
    end
-->

3 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅∑((i), x(i)⋅b(i))+0.5⋅xᵀ⋅A⋅x)
        𝒫((a, b) -> 
            E((i) -> 
  

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        𝒫((a, b) -> 
            E((a⋅(b⋅p+r)+x)))
    end

In [12]:
df = vdiff(f) |> blaserize 

1 sum_out
∑((i), a⋅p(i)⋅𝒫(E)((i) -> 
    (a⋅(b⋅p(i)+r(i))+x(i)), 1)(i))
Mul
∑((i), p(i)⋅𝒫(E)((i) -> 
    (a⋅(b⋅p(i)+r(i))+x(i)), 1)(i))⋅a
-->

1 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((i, j), 0.5⋅x(i)⋅x(j)⋅A(i, j))-1.0⋅∑((i), x(i)⋅b(i)))
        (a, b) -> 
            ∑((i), (b⋅p(i)+r(i))⋅𝒫(E)((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i)), 1)(i)), ∑((i), p(i)⋅𝒫(E)((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i)), 1)(i))⋅a
    end
Map
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((j), 0.5⋅(xᵀ⋅A)(j)⋅x(j))-1.0⋅∑((i), x(i)⋅b(i)))
        (a, b) -> 
            ∑((i), (b⋅p(i)+r(i))⋅𝒫(E)((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i)), 1)(i)), ∑((i), p(i)⋅𝒫(E)((i) -> 
                (a⋅(b⋅p(i)+r(i))+x(i)), 1)(i))⋅a
    end
-->

2 blas mul
(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (∑((j), 0.5⋅(xᵀ⋅A)(j)⋅x(j))-1.0⋅∑((i), x(i)⋅b(i)))
        (a, b) -> 
            ∑((i), (b⋅p(i)+r(i))⋅𝒫(E)((i) -> 
                (a⋅(b⋅p(i)+r(

(A, r, p, b, x) -> 
    let 
        E = (x) -> 
            (-1.0⋅xᵀ⋅b+0.5⋅xᵀ⋅A⋅x)
        (a, b) -> 
            𝒫(E)((a⋅(b⋅p+r)+x), 1)ᵀ⋅(b⋅p+r), 𝒫(E)((a⋅(b⋅p+r)+x), 1)ᵀ⋅p⋅a
    end

In [13]:
df = redux(vdiff(eval_all(f)); settings=symmetry_settings) |> blaserize # |> latex |> println

1 sum_out
(∑((_z, i), -1.0⋅δ(∑((i, j), 0.5⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j)), _z, 1)⋅(_b⋅p(i)+r(i))⋅b(i))+∑((i, j), (∑((_z_1), 0.5⋅δ((a⋅(_b⋅p(i)+r(i))+x(i)), _z_1, 1)⋅_z_1⋅(_b⋅p(j)+r(j))⋅A(i, j))+0.5⋅(_b⋅p(i)+r(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j))))
Add
(∑((i, j), (∑((_z_1), 0.5⋅δ((a⋅(_b⋅p(i)+r(i))+x(i)), _z_1, 1)⋅_z_1⋅(_b⋅p(j)+r(j))⋅A(i, j))+0.5⋅(_b⋅p(i)+r(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j)))-1.0⋅∑((_z, i), δ(∑((i, j), 0.5⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j)), _z, 1)⋅(_b⋅p(i)+r(i))⋅b(i)))
-->

2 sum_dist
(∑((i, j), (∑((_z_1), 0.5⋅δ((a⋅(_b⋅p(i)+r(i))+x(i)), _z_1, 1)⋅_z_1⋅(_b⋅p(j)+r(j))⋅A(i, j))+0.5⋅(_b⋅p(i)+r(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j)))-1.0⋅∑((_z, i), δ(∑((i, j), 0.5⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j)), _z, 1)⋅(_b⋅p(i)+r(i))⋅b(i)))
Add
(∑((i, j), 0.5⋅(_b⋅p(i)+r(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j))+∑((i, j, _z_1), 0.5⋅δ((a⋅(_b⋅p(i)+r(i))+x(i)), _z_1, 1)⋅_z_1⋅(_b⋅p(j)+r(j))⋅A(i, j))-1.0⋅∑((_z, i), δ(∑((i, j), 0.5⋅

1 symmetry
(-1.0⋅∑((i), (_b⋅p(i)+r(i))⋅b(i))+0.5⋅(∑((i, j), (_b⋅p(i)+r(i))⋅(a⋅(_b⋅p(j)+r(j))+x(j))⋅A(i, j))+∑((i, j), (_b⋅p(j)+r(j))⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅A(i, j)))), a⋅(-1.0⋅∑((i), b(i)⋅p(i))+0.5⋅(∑((i, j), (a⋅(_b⋅p(i)+r(i))+x(i))⋅p(j)⋅A(i, j))+∑((i, j), (a⋅(_b⋅p(j)+r(j))+x(j))⋅p(i)⋅A(i, j))))
PCTVector
(∑((i, j), (_b⋅p(j)+r(j))⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅A(i, j))-1.0⋅∑((i), (_b⋅p(i)+r(i))⋅b(i))), a⋅(-1.0⋅∑((i), b(i)⋅p(i))+0.5⋅(∑((i, j), (a⋅(_b⋅p(i)+r(i))+x(i))⋅p(j)⋅A(i, j))+∑((i, j), (a⋅(_b⋅p(j)+r(j))+x(j))⋅p(i)⋅A(i, j))))
-->

2 symmetry
(∑((i, j), (_b⋅p(j)+r(j))⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅A(i, j))-1.0⋅∑((i), (_b⋅p(i)+r(i))⋅b(i))), a⋅(-1.0⋅∑((i), b(i)⋅p(i))+0.5⋅(∑((i, j), (a⋅(_b⋅p(i)+r(i))+x(i))⋅p(j)⋅A(i, j))+∑((i, j), (a⋅(_b⋅p(j)+r(j))+x(j))⋅p(i)⋅A(i, j))))
PCTVector
(∑((i, j), (_b⋅p(j)+r(j))⋅(a⋅(_b⋅p(i)+r(i))+x(i))⋅A(i, j))-1.0⋅∑((i), (_b⋅p(i)+r(i))⋅b(i))), a⋅(∑((i, j), (a⋅(_b⋅p(j)+r(j))+x(j))⋅p(i)⋅A(i, j))-1.0⋅∑((i), b(i)⋅p(i)))
-->

3 symmetry
(∑((i, j), (_b⋅p(j)+r(j))⋅(a⋅(_b⋅p(i)+r

(A, r, p, b, x) -> 
    (a, _b) -> 
        ((a⋅(_b⋅p+r)+x)ᵀ⋅A⋅(_b⋅p+r)-1.0⋅(_b⋅p+r)ᵀ⋅b), a⋅(pᵀ⋅A⋅(a⋅(_b⋅p+r)+x)-1.0⋅bᵀ⋅p)

In [14]:
_, ctx = @pct begin    
    @space T begin
        type = (N, N, N, N) -> C
        symmetries = (((2, 1, 4, 3), :conj), ((3, 4, 1, 2), :id))
    end
end;

In [15]:
f, _ = @pct _ ctx (J::T) -> pullback((C::CM) -> sum((i, j, p, q, r, s),
    C(p, i)' * C(q, i) * C(r, j)' * C(s, j) * J(p, q, r, s))); f

(J) -> 
    𝒫((C) -> 
        ∑((i, j, p, q, r, s), C(p, i)'⋅C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))

In [16]:
df = redux(vdiff(f); settings=symmetry_settings)

1 sum_dist
∑((i, j, p, q, r, s), ((_d, _d_1) -> 
    δ(i, _d_1, δ(p, _d, C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))+∑((_z_1), ((_d, _d_1) -> 
    δ(j, _d_1, δ(r, _d, δ(_z_1, C(p, i)', 1)⋅_z_1⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))+∑((_z_2), ((_d, _d_1) -> 
    δ(j, _d_1, δ(s, _d, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))⋅_z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'))+(_d, _d_1) -> 
    ∑((_z_3), δ(i, _d_1, δ(q, _d, δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))⋅_z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)')))))))))
Add
((_d, _d_1) -> 
    ∑((i, j, p, q, r, s), δ(i, _d_1, δ(p, _d, C(r, j)'⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i))))+∑((i, j, p, q, r, s, _z_1), ((_d, _d_1) -> 
    δ(j, _d_1, δ(r, _d, δ(_z_1, C(p, i)', 1)⋅_z_1⋅J(p, q, r, s)⋅C(s, j)⋅C(q, i)))+∑((_z_2), ((_d, _d_1) -> 
    δ(j, _d_1, δ(s, _d, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))⋅_z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'))+(_d, _d_1) -> 
    ∑((_z_3), δ(i, _d_1, δ(q, _d, δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))⋅_z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'

10 sum_dist
((_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1)))+∑((i, j, p, q, r, s, _z_1, _z_2), ((_d, _d_1) -> 
    δ(j, _d_1, δ(s, _d, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))⋅_z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)'))+(_d, _d_1) -> 
    ∑((_z_3), δ(i, _d_1, δ(q, _d, δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))⋅_z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)'))))))
Add
((_d, _d_1) -> 
    ∑((i, j, p, q, r, s, _z_1, _z_2), δ(j, _d_1, δ(s, _d, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1))⋅_z_2'⋅_z_1'⋅J(p, q, r, s)'⋅C(q, i)')))+(_d, _d_1) -> 
    ∑((i, j, p, q, r, s, _z_1, _z_2, _z_3), δ(i, _d_1, δ(q, _d, δ(C(s, j), _z_3, δ(C(r, j)', _z_2, δ(C(p, i)', _z_1, 1)))⋅_z_2'⋅_z_3'⋅_z_1'⋅J(p, q, r, s)')))+(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))))
-->

11 combine map
((_d, _d_1) -> 
    ∑((i, j, p, q, r, s, _z_1, _z_2), δ(

21 contract_delta
(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, p, q, r), J(p, q, r, _d)'⋅C(q, i)'⋅C(p, i)⋅C(r, _d_1))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, _z_2, _z_3, j, p, r, s), δ(_z_3, C(s, j), δ(C(r, j)', _z_2, δ(C(p, _d_1)', _z_1, 1))⋅_z_2'⋅_z_3'⋅_z_1'⋅J(p, _d, r, s)')))
Map
(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, p, q, r), J(p, q, r, _d)'⋅C(q, i)'⋅C(p, i)⋅C(r, _d_1))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, _z_2, j, p, r, s), δ(C(r, j)', _z_2, δ(C(p, _d_1)', _z_1, 1))⋅_z_2'⋅_z_1'⋅J(p, _d, r, s)'⋅C(s, j)'))
-->

22 swallow
(_d, _d_1) -> 
    (∑((i, p, q, s), C(p, i)'⋅J(p, q, _d, s)⋅C(s, _d_1)⋅C(q, i))+∑((i, p, q, r), J(p, q, r, _d)'⋅C(q, i)'⋅C(p, i)⋅C(r, _d_1))+∑((j, q, r, s), C(r, j)'⋅J(_d, q, r, s)⋅C(s, j)⋅C(q, _d_1))+∑((_z_1, _z_2, j, p, r, s), δ(C(r, j)', _z_2, δ(C(p, _d_1)', _z_1, 1))⋅_z_2'⋅_z_1'⋅J(p, _d, r, s)'⋅C(s, j)'))
Map
(_d,

(J) -> 
    (C) -> 
        (_d, _d_1) -> 
            4.0⋅∑((j, p, r, s), C(s, j)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j))

In [17]:
get_body(df) |> decompose |> pp |> simplify |> first

1 sum_dist
∑((_d, _d_1, j, p, r, s), ((_d_2, _d_3) -> 
    δ(j, _d_3, δ(s, _d_2, 4.0⋅_k(_d, _d_1)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j)))+∑((_z_1), ((_d_3, _d_2) -> 
    δ(_d_1, _d_2, δ(p, _d_3, δ(C(s, j)', _z_1, 4.0⋅_k(_d, _d_1))⋅_z_1'⋅J(_d, p, s, r)'⋅C(r, j)'))+(_d_2, _d_3) -> 
    ∑((_z_2), δ(j, _d_3, δ(r, _d_2, δ(C(p, _d_1), _z_2, δ(C(s, j)', _z_1, 4.0⋅_k(_d, _d_1)))⋅_z_2'⋅_z_1'⋅J(_d, p, s, r)')))))))
Add
((_d_2, _d_3) -> 
    ∑((_d, _d_1, j, p, r, s), δ(j, _d_3, δ(s, _d_2, 4.0⋅_k(_d, _d_1)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j))))+∑((_d, _d_1, j, p, r, s, _z_1), ((_d_3, _d_2) -> 
    δ(_d_1, _d_2, δ(p, _d_3, δ(C(s, j)', _z_1, 4.0⋅_k(_d, _d_1))⋅_z_1'⋅J(_d, p, s, r)'⋅C(r, j)'))+(_d_2, _d_3) -> 
    ∑((_z_2), δ(j, _d_3, δ(r, _d_2, δ(C(p, _d_1), _z_2, δ(C(s, j)', _z_1, 4.0⋅_k(_d, _d_1)))⋅_z_2'⋅_z_1'⋅J(_d, p, s, r)'))))))
-->

2 contract_delta
((_d_2, _d_3) -> 
    ∑((_d, _d_1, j, p, r, s), δ(j, _d_3, δ(s, _d_2, 4.0⋅_k(_d, _d_1)'⋅J(_d, p, s, r)⋅C(p, _d_1)⋅C(r, j))))+∑((_d, _d_1, j, p, r, s, _

13 contract_delta
(_a, _a_1) -> 
    (∑((_d, _d_1, j, p, r, s, _z_1, _z_2), δ(j, _a_1, δ(r, _a, δ(C(p, _d_1), _z_2, δ(C(s, j)', _z_1, 4.0⋅_k(_d, _d_1)))⋅_z_2'⋅_z_1'⋅J(_d, p, s, r)')))+4.0⋅(∑((_d, _d_1, p, r), _k(_d, _d_1)'⋅J(_d, p, _a, r)⋅C(p, _d_1)⋅C(r, _a_1))+∑((_d, j, r, s), J(_d, _a, s, r)'⋅C(r, j)'⋅_k(_d, _a_1)⋅C(s, j))))
Map
(_a, _a_1) -> 
    (∑((_d, _d_1, _z_1, _z_2, p, r, s), δ(r, _a, δ(C(p, _d_1), _z_2, δ(C(s, _a_1)', _z_1, 4.0⋅_k(_d, _d_1)))⋅_z_2'⋅_z_1'⋅J(_d, p, s, r)'))+4.0⋅(∑((_d, _d_1, p, r), _k(_d, _d_1)'⋅J(_d, p, _a, r)⋅C(p, _d_1)⋅C(r, _a_1))+∑((_d, j, r, s), J(_d, _a, s, r)'⋅C(r, j)'⋅_k(_d, _a_1)⋅C(s, j))))
-->

14 contract_delta
(_a, _a_1) -> 
    (∑((_d, _d_1, _z_1, _z_2, p, r, s), δ(r, _a, δ(C(p, _d_1), _z_2, δ(C(s, _a_1)', _z_1, 4.0⋅_k(_d, _d_1)))⋅_z_2'⋅_z_1'⋅J(_d, p, s, r)'))+4.0⋅(∑((_d, _d_1, p, r), _k(_d, _d_1)'⋅J(_d, p, _a, r)⋅C(p, _d_1)⋅C(r, _a_1))+∑((_d, j, r, s), J(_d, _a, s, r)'⋅C(r, j)'⋅_k(_d, _a_1)⋅C(s, j))))
Map
(_a, _a_1) -> 
    (∑((_d, _d_1, _z_1, _z_

(C, _k) -> 
    (_a, _a_1) -> 
        4.0⋅(∑((_d, _d_1, p, s), J(_d, p, s, _a)'⋅C(p, _d_1)'⋅_k(_d, _d_1)⋅C(s, _a_1))+∑((_d, _d_1, p, r), _k(_d, _d_1)'⋅J(_d, p, _a, r)⋅C(p, _d_1)⋅C(r, _a_1))+∑((_d, j, r, s), J(_d, _a, s, r)'⋅C(r, j)'⋅_k(_d, _a_1)⋅C(s, j)))

In [18]:
_, ctx = @pct begin
    @domain BZ begin
        base = I
        periodic = true
    end
    
    @domain X begin
        symmetric = true
        contractable = false
    end
    
    @space Mmn begin
        type = (N, N, BZ, BZ) -> C
        symmetries = (((2, 1, 4, 3), :conj),)
    end

    @space SV begin
        type = (I,) -> R
        symmetries = (((1,), :ineg),)
    end
    
    @space Gauge begin
        type = (N, N, BZ) -> C
    end
end;

In [19]:
f, _ = @pct _ ctx (S::Mmn, w::SV) -> pullback((U::Gauge) -> 
    begin
        ρ = (n::N, b::X) -> sum((k::BZ, p, q), U(p, n, k)' * S(p, q, k, k + b) * U(q, n, k + b))
        sum((n, b::X), w(b) * ρ(n, b)' * ρ(n, b))
    end); f

(S, w) -> 
    𝒫((U) -> 
        let 
            ρ = (n, b) -> 
                ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))
            ∑((n, b), ρ(n, b)'⋅w(b)⋅ρ(n, b))
        end)

In [20]:
df = vdiff(eval_all(f))

1 sum_dist
∑((n, b), (∑((k, p, q), ((_d, _d_1, _d_2) -> 
    ∑((_z_1), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(U(p, n, k)', _z_1, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))⋅w(b))⋅_z_1'⋅S(p, q, k, (b+k))'))))+(_d, _d_1, _d_2) -> 
    δ(k, _d_2, δ(n, _d_1, δ(p, _d, ∑((k, p, q), U(q, n, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, n, k))⋅w(b)⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))))))+∑((_z_3, k, p, q), ((_d, _d_1, _d_2) -> 
    ∑((_z_2), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(_z_2, U(p, n, k), δ(_z_3, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), 1)⋅_z_3⋅w(b))⋅_z_2⋅S(p, q, k, (b+k))'))))+(_d, _d_1, _d_2) -> 
    δ(k, _d_2, δ(n, _d_1, δ(p, _d, δ(∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), _z_3, 1)⋅_z_3'⋅w(b)⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))))))))
Add
(∑((n, b, k, p, q), ((_d, _d_1, _d_2) -> 
    ∑((_z_1), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(U(p, n, k)', _z_1, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))⋅w(b))⋅_z_1'⋅S(p, q, k, (b+k))'))))+(_d, _d_1, _

7 sum_in
((_d, _d_1, _d_2) -> 
    (∑((b, q), ∑((k, p, q), U(q, _d_1, (b+k))'⋅S(p, q, k, (b+k))'⋅U(p, _d_1, k))⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅S(_d, q, _d_2, (_d_2+b)))+∑((n, b, k, p, q, _z_1), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(U(p, n, k)', _z_1, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k)))⋅w(b))⋅_z_1'⋅S(p, q, k, (b+k))')))))+∑((n, b, _z_3, k, p, q), ((_d, _d_1, _d_2) -> 
    ∑((_z_2), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(_z_2, U(p, n, k), δ(_z_3, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), 1)⋅_z_3⋅w(b))⋅_z_2⋅S(p, q, k, (b+k))'))))+(_d, _d_1, _d_2) -> 
    δ(k, _d_2, δ(n, _d_1, δ(p, _d, δ(∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), _z_3, 1)⋅_z_3'⋅w(b)⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))
Add
((_d, _d_1, _d_2) -> 
    (∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b)))+∑((n, b, k, p, q, _z_1), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(U(p, n, k)', _z_1, ∑((k, p, q), U(p, n

13 sum_in
((_d, _d_1, _d_2) -> 
    (∑((b, p), ∑((k, _p, q), U(_p, _d_1, k)'⋅U(q, _d_1, (b+k))⋅S(_p, q, k, (b+k)))⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(p, _d_1, (-1.0⋅b+_d_2)))+∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b))))+∑((n, b, _z_3, k, p, q), ((_d, _d_1, _d_2) -> 
    ∑((_z_2), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(_z_2, U(p, n, k), δ(_z_3, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), 1)⋅_z_3⋅w(b))⋅_z_2⋅S(p, q, k, (b+k))'))))+(_d, _d_1, _d_2) -> 
    δ(k, _d_2, δ(n, _d_1, δ(p, _d, δ(∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), _z_3, 1)⋅_z_3'⋅w(b)⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))))))))
Add
((_d, _d_1, _d_2) -> 
    (∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k)))

20 contract_delta
(_d, _d_1, _d_2) -> 
    (∑((_z_3, b, q), δ(_z_3, ∑((k, p, q), U(p, _d_1, k)'⋅U(q, _d_1, (b+k))⋅S(p, q, k, (b+k))), _z_3'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅S(_d, q, _d_2, (_d_2+b))))+∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k)))+∑((n, b, _z_3, k, p, q, _z_2), δ((b+k), _d_2, δ(n, _d_1, δ(q, _d, δ(_z_2, U(p, n, k), δ(_z_3, ∑((k, p, q), U(p, n, k)'⋅U(q, n, (b+k))⋅S(p, q, k, (b+k))), 1)⋅_z_3⋅w(b))⋅_z_2⋅S(p, q, k, (b+k))')))))
Map
(_d, _d_1, _d_2) -> 
    (∑((b, q), ∑((k, p, q), U(p, _d_1, k)⋅U(q, _d_1, (b+k))'⋅S(p, q, k, (b+k))')⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2

28 contract_delta
(_d, _d_1, _d_2) -> 
    (∑((_z_3, b, p), δ(∑((k, _p, q), U(_p, _d_1, k)'⋅U(q, _d_1, (b+k))⋅S(_p, q, k, (b+k))), _z_3, _z_3⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(p, _d_1, (-1.0⋅b+_d_2))))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k)))+2.0⋅∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b))))
Map
(_d, _d_1, _d_2) -> 
    (∑((b, p), ∑((k, _p, q), U(_p, _d_1, k)'⋅U(q, _d_1, (b+k))⋅S(_p, q, k, (b+k)))⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(p, _d_1, (-1.0⋅b+_d_2)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k)))+2.0⋅∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b))))
-->

29 sum_in
(_d, _d_1, _d_2) -> 
    (∑((b, p), ∑((k, _p, q), U(_p, _d_1, k)'⋅U(q, _d_

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            2.0⋅(∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k))))

In [21]:
redux(df; settings=symmetry_settings)

1 shift
2.0⋅(∑((b, q, k, p, _i), U(_i, _d_1, (b+k))'⋅S(p, _i, k, (b+k))'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, k)⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k))))
Mul
2.0⋅(∑((b, q, k, p, _i), U(_i, _d_1, k)'⋅S(p, _i, (k-1.0⋅b), k)'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, (k-1.0⋅b))⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k))))
<->

2 sum_sym
2.0⋅(∑((b, q, k, p, _i), U(_i, _d_1, k)'⋅S(p, _i, (k-1.0⋅b), k)'⋅w(b)⋅U(q, _d_1, (_d_2+b))⋅U(p, _d_1, (k-1.0⋅b))⋅S(_d, q, _d_2, (_d_2+b)))+∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (-1.0⋅b+_d_2), _d_2)'⋅w(b)⋅U(q, _d_1, (b+k))⋅U(p, _d_1, (-1.0⋅b+_d_2))⋅S(_p, q, k, (b+k))))
Mul
2.0⋅(∑((b, q, k, p, _i), U(_i, _d_1, k)'⋅S(p, _i, (b+k), k)'⋅w(-1.0⋅b)⋅U(q, _d_1, (-1.0⋅b+_d_2))⋅U(p, _d_1, (b+k))⋅S(_d, q, _d_2, (-1.0⋅b+_d_2)))+∑((

22 shift
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
<->

23 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
<->

24 shift
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, (b+k), k))
<->

25 sum_sym
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, (b+k

51 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(q, _p, (k-1.0⋅b), k)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(q, _p, (k-1.0⋅b), k)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b)))
<->

52 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(q, _p, (k-1.0⋅b), k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(q, _p, (k-1.0⋅b), k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b)))
<->

53 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅S(q, _p, k, (b+k))'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅S(q, _p, k, (b+k))'⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b)))
<->

54 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅S(q, _p, k, (b+k))'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, 

79 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(q, _p, (k-1.0⋅b), k)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b)))
<->

80 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
<->

81 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b))⋅S(_p, q, k, (k-1.0⋅b)))
Mul
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, k)'⋅S(q, _p, (k-1.0⋅b), k)'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(-1.0⋅b)⋅U(q, _d_1, (k-1.0⋅b))⋅U(p, _d_1, (_d_2+b)))
<->

82 symmetry
4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅S(p, _d, (_d_2+b), _d_2)'⋅w(

(S, w) -> 
    (U) -> 
        (_d, _d_1, _d_2) -> 
            4.0⋅∑((b, p, k, _p, q), U(_p, _d_1, (b+k))'⋅w(b)⋅U(q, _d_1, k)⋅U(p, _d_1, (_d_2+b))⋅S(_d, p, _d_2, (_d_2+b))⋅S(_p, q, (b+k), k))